In [1]:
# Import cell, ensure this runs first!
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
DATA_PATH = "../data/used/"

# Loading the kaggle data
KAGGLE_SUFFIX = ".us.txt"   # All the kaggle-gathered data ends with this file type, convenience variable.
BIL  = pd.read_csv(os.path.join(DATA_PATH, "bil" + KAGGLE_SUFFIX), parse_dates=["Date"])
BND  = pd.read_csv(os.path.join(DATA_PATH, "bnd" + KAGGLE_SUFFIX), parse_dates=["Date"])
EFA  = pd.read_csv(os.path.join(DATA_PATH, "efa" + KAGGLE_SUFFIX), parse_dates=["Date"])
RWR  = pd.read_csv(os.path.join(DATA_PATH, "rwr" + KAGGLE_SUFFIX), parse_dates=["Date"])
SPY  = pd.read_csv(os.path.join(DATA_PATH, "spy" + KAGGLE_SUFFIX), parse_dates=["Date"])
VNQ  = pd.read_csv(os.path.join(DATA_PATH, "vnq" + KAGGLE_SUFFIX), parse_dates=["Date"])
VTI  = pd.read_csv(os.path.join(DATA_PATH, "vti" + KAGGLE_SUFFIX), parse_dates=["Date"])
VXUS = pd.read_csv(os.path.join(DATA_PATH, "vxus" + KAGGLE_SUFFIX), parse_dates=["Date"]) # Kaggle data only has ~1700 rows

# ============== Loading other data sources ==============
# From Federal Reserve Bank of St. Louis, https://fred.stlouisfed.org/series/DTB3
DTB3 = pd.read_csv(os.path.join(DATA_PATH, "DTB3.csv"), parse_dates=["observation_date"])
DTB3.rename(columns={"observation_date": "Date"}, inplace=True)

# From fetch_yahoo.py script
# yfinance data has odd formatting, need to reformat before parsing date column.
VBMFX = pd.read_csv(os.path.join(DATA_PATH, "vbmfx.csv"), header=None)

col_names = VBMFX.iloc[0].tolist()

col_names[0] = "Date"
tckr = VBMFX.iloc[1, 1]

VBMFX = VBMFX.iloc[3:].copy()
VBMFX.columns = col_names

VBMFX["Date"] = pd.to_datetime(VBMFX["Date"])

BIL.set_index("Date", inplace=True)
BND.set_index("Date", inplace=True)
DTB3.set_index("Date", inplace=True)
EFA.set_index("Date", inplace=True)
RWR.set_index("Date", inplace=True)
SPY.set_index("Date", inplace=True)
VBMFX.set_index("Date", inplace=True)
VNQ.set_index("Date", inplace=True)
VTI.set_index("Date", inplace=True)
VXUS.set_index("Date", inplace=True)

After some initial exploration, I found that some of the kaggle data had relatively few observations (~1700) while other data points had many observations (>3000). For now, I will continue exploring the data set as is, because my yahoo data is rate-limited, but long term I would like to pull down more data from yahoo finance, contrast it with my initial dataset, and potentially centralize my dataset around yahoo finance, for a longer horizon. For now, my Markowitz Optimization model will be highly tuned towards modern economic patterns.

In [3]:
print(f"Total NA in BIL  : {BIL.isna().sum()}")
print(f"Total NA in BND  : {BND.isna().sum()}")
print(f"Total NA in DTB3 : {DTB3.isna().sum()}")
print(f"Total NA in EFA  : {EFA.isna().sum()}")
print(f"Total NA in RWR  : {RWR.isna().sum()}")
print(f"Total NA in SPY  : {SPY.isna().sum()}")
print(f"Total NA in VBMFX: {VBMFX.isna().sum()}")
print(f"Total NA in VNQ  : {VNQ.isna().sum()}")
print(f"Total NA in VTI  : {VTI.isna().sum()}")
print(f"Total NA in VXUS : {VXUS.isna().sum()}")

Total NA in BIL  : Open       0
High       0
Low        0
Close      0
Volume     0
OpenInt    0
dtype: int64
Total NA in BND  : Open       0
High       0
Low        0
Close      0
Volume     0
OpenInt    0
dtype: int64
Total NA in DTB3 : DTB3    795
dtype: int64
Total NA in EFA  : Open       0
High       0
Low        0
Close      0
Volume     0
OpenInt    0
dtype: int64
Total NA in RWR  : Open       0
High       0
Low        0
Close      0
Volume     0
OpenInt    0
dtype: int64
Total NA in SPY  : Open       0
High       0
Low        0
Close      0
Volume     0
OpenInt    0
dtype: int64
Total NA in VBMFX: Close     1
High      1
Low       1
Open      1
Volume    0
dtype: int64
Total NA in VNQ  : Open       0
High       0
Low        0
Close      0
Volume     0
OpenInt    0
dtype: int64
Total NA in VTI  : Open       0
High       0
Low        0
Close      0
Volume     0
OpenInt    0
dtype: int64
Total NA in VXUS : Open       0
High       0
Low        0
Close      0
Volume     0
OpenInt   

Only the DTB3 data set has missing data

In [4]:
close_vals =  [
                BIL["Close"]  , BND["Close"],
                DTB3["DTB3"]  , EFA["Close"],
                RWR["Close"]  , SPY["Close"],
                VBMFX["Close"], VNQ["Close"],
                VTI["Close"]  , VXUS["Close"],
              ]

col_names = ["BIL", "BND", "DTB3", "EFA", "RWR", "SPY", "VBMFX", "VNQ", "VTI", "VXUS"]
combined = pd.concat(close_vals, axis=1)
combined.columns = col_names

combined.sort_index(inplace=True)
cleaned = combined.dropna(how="any")  # Find the max period where no feature has an NA

cleaned[col_names] = cleaned[col_names].apply(pd.to_numeric)

C:\Users\drewm\AppData\Local\Temp\ipykernel_5592\1920781367.py:10: Pandas4Warning: Sorting by default when concatenating all DatetimeIndex is deprecated.  In the future, pandas will respect the default of `sort=False`. Specify `sort=True` or `sort=False` to silence this message. If you see this warnings when not directly calling concat, report a bug to pandas.
  combined = pd.concat(close_vals, axis=1)


It looks like our dataset has a complete and consistent daily close price for each of our selected stocks, for a period stretching from  2011-02-01 to 2017-11-10, roughly 6.5 years. My goal of Markowitz Optimization can be significantly skewed by time, policies such as ZIRP can significantly change the relationships between features studied in this project, altering the mathematical optimum. As such, it is ideal for this project to collect further data enabling a window closer to the modern day, but this is sufficient for now.

`VXUS`, `BIL`, and `BND` have the smallest number of observations (`VXUS` ~1700, `BIL/BND` ~2600)

The `cleaned` dataframe now contains date-aligned, frequency-matched, gapless data for all of the features used in this project.

In [6]:
ret = cleaned.iloc[1:].pct_change() # Simple daily returns

means = []
for idx, col in enumerate(list(ret.columns)):
  means.append(ret[col].mean())

c:\Users\drewm\miniforge3\envs\capstone\Lib\site-packages\numpy\_core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
